In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import time
%matplotlib inline

In [2]:
from nba_py.team import TeamYearOverYearSplits, TeamList
team_list = TeamList().info().head(30)

In [3]:
rockets = TeamYearOverYearSplits(1610612745).by_year()
rockets.head(1)

,GROUP_SET,GROUP_VALUE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
0,By Year,2017-18,72,58,14,0.806,48.2,38.9,84.3,0.462,...,3,3,11,6,7,7,2,1,210,2017-18


In [4]:
season_team = {}
for team in team_list['TEAM_ID']:
    df = TeamYearOverYearSplits(team,season_type='Playoffs').by_year()
    for index, row in df.iterrows():
        season_data =  season_team.get(row['GROUP_VALUE'])
        if season_data:
            if team not in season_team[row['GROUP_VALUE']]:
                season_team[row['GROUP_VALUE']].append(team)
        else:
            season_team[row['GROUP_VALUE']] = [team]
    time.sleep(2)

In [5]:
def playoff_team(team_id, season):
    if team_id in season_team[season]:
        return 1
    return 0

In [6]:
all_team_data = pd.DataFrame()
for team in team_list['TEAM_ID']:
    team_data = TeamYearOverYearSplits(team,measure_type = 'Advanced').by_year()
    team_data['PLAYOFFS'] = team_data.apply(lambda row: playoff_team(team,row['GROUP_VALUE']),axis=1)
    all_team_data = pd.concat([all_team_data,team_data])
    time.sleep(2)

KeyError: ('2017-18', 'occurred at index 0')

In [14]:
all_team_data = pd.DataFrame()
for team in team_list['TEAM_ID']:
    team_data = TeamYearOverYearSplits(team).by_year()
    team_data['PLAYOFFS'] = team_data.apply(lambda row: playoff_team(team,row['GROUP_VALUE']),axis=1)
    all_team_data = pd.concat([all_team_data,team_data])
    time.sleep(2)

In [7]:
regular_stats = pd.read_csv('all_team_playoffs.csv')
advs_stats = pd.read_csv('all_team_playoffs_adv.csv')

In [8]:
regular_features = regular_stats[['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA',
       'PF', 'PFD', 'PTS', 'PLUS_MINUS']]

In [9]:
advs_features = advs_stats[['NET_RATING', 'AST_PCT', 'AST_TO',
       'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT',
       'TS_PCT', 'PACE', 'PIE']]

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(advs_features, advs_stats['PLAYOFFS'], test_size=0.30, random_state=69)

In [73]:
from sklearn.preprocessing import StandardScaler

In [74]:
scaler = StandardScaler()

In [75]:
scaler.fit(advs_features)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [76]:
scaled_features = scaler.fit_transform(advs_features)

In [78]:
df_advs_feat = pd.DataFrame(scaled_features,columns = advs_features.columns)
df_advs_feat.head()

,NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,PACE,PIE
0,-1.049373,0.756361,0.159141,0.787408,-1.676275,0.976510,-0.997667,0.166296,1.006109,0.875533,1.846938,-0.816897
1,-0.160531,0.732619,0.051506,0.534963,-1.092083,1.152907,0.195143,0.243303,0.571482,0.358077,1.646540,0.167763
2,0.807320,1.729787,1.181674,1.881339,-2.475697,0.623718,-1.660339,-0.295743,1.093034,0.875533,1.511898,0.786693
3,1.103600,2.038434,1.719850,2.386230,-1.768516,0.200367,-1.196468,-0.526762,1.571124,1.392988,0.525568,1.124291
4,-0.140779,1.824755,0.751134,1.797190,-1.891504,0.553159,-1.395270,0.166296,1.049572,0.969616,0.719703,0.055231


In [79]:
X = df_advs_feat

In [80]:
y = advs_stats['PLAYOFFS']

In [81]:
X = X.as_matrix()
y = y.as_matrix()

In [82]:
from sklearn.cross_validation import train_test_split

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [84]:
import tensorflow.contrib.learn.python.learn as learn

ModuleNotFoundError: No module named 'tensorflow'

In [63]:
from sklearn.ensemble import RandomForestClassifier

In [64]:
rfc = RandomForestClassifier(n_estimators = 600)

In [70]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=600, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [71]:
rfc_pred = rfc.predict(X_test)

In [72]:
#rfc with advanced stats
print(classification_report(y_test,rfc_pred))

             precision    recall  f1-score   support

          0       0.87      0.89      0.88        87
          1       0.91      0.89      0.90       109

avg / total       0.89      0.89      0.89       196



In [68]:
#rfc with regular stats
print(classification_report(y_test,rfc_pred))

             precision    recall  f1-score   support

          0       0.85      0.89      0.87        87
          1       0.90      0.87      0.89       109

avg / total       0.88      0.88      0.88       196



In [53]:
from sklearn.tree import DecisionTreeClassifier

In [59]:
dtree = DecisionTreeClassifier()

In [60]:
dtree.fit(X_train,y_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [61]:
dtree_pred = dtree.predict(X_test)

In [57]:
#dtree with advanced stats
print(classification_report(y_test,dtree_pred))

             precision    recall  f1-score   support

          0       0.82      0.89      0.85        87
          1       0.90      0.84      0.87       109

avg / total       0.87      0.86      0.86       196



In [62]:
#dtree with reuglar stats
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.84      0.90      0.87        87
          1       0.91      0.86      0.89       109

avg / total       0.88      0.88      0.88       196



In [40]:
from sklearn.svm import SVC

In [47]:
svc_model = SVC()

In [48]:
svc_model.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [49]:
predictions = svc_model.predict(X_test)

In [51]:
#classification report for support vector machine

In [50]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.84      0.90      0.87        87
          1       0.91      0.86      0.89       109

avg / total       0.88      0.88      0.88       196



In [11]:
from sklearn.linear_model import LogisticRegression
regular_model = LogisticRegression()
regular_model.fit(X_train, y_train)
predictions = regular_model.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.86      0.90      0.88        87
          1       0.91      0.88      0.90       109

avg / total       0.89      0.89      0.89       196



In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(advs_features, advs_stats['PLAYOFFS'], test_size=0.30, random_state=300)

In [13]:
adv_model = LogisticRegression()
adv_model.fit(X_train, y_train)
predictions = adv_model.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.87      0.95      0.91        96
          1       0.95      0.86      0.90       100

avg / total       0.91      0.90      0.90       196



In [14]:
test_team =  TeamYearOverYearSplits(1610612740).by_year()[['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA',
       'PF', 'PFD', 'PTS', 'PLUS_MINUS']]
logmodel.predict(test_team.head(1))

NameError: name 'logmodel' is not defined

In [15]:
current_predictions_norm = {}
current_predictions_adv = {}
for index, row in team_list.iterrows():
    current = TeamYearOverYearSplits(row['TEAM_ID']).by_year()[['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA',
       'PF', 'PFD', 'PTS', 'PLUS_MINUS']]
    current_adv = TeamYearOverYearSplits(row['TEAM_ID'],measure_type = 'Advanced').by_year()[['NET_RATING', 'AST_PCT', 'AST_TO',
       'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT',
       'TS_PCT', 'PACE', 'PIE']]
    current_predictions_norm[row['ABBREVIATION']] = regular_model.predict(current.head(1))
    current_predictions_adv[row['ABBREVIATION']] = adv_model.predict(current_adv.head(1))

In [16]:
for norm, adv in zip(current_predictions_norm.keys(),current_predictions_adv.keys()):
    print("Normal Prediciton: "+norm+" "+str(current_predictions_norm[norm]))
    print("Advanced Prediciton: "+adv+" "+str(current_predictions_adv[adv]))
    print("------------------------------")

Normal Prediciton: ATL [0]
Advanced Prediciton: ATL [0]
------------------------------
Normal Prediciton: BOS [1]
Advanced Prediciton: BOS [1]
------------------------------
Normal Prediciton: CLE [0]
Advanced Prediciton: CLE [1]
------------------------------
Normal Prediciton: NOP [0]
Advanced Prediciton: NOP [1]
------------------------------
Normal Prediciton: CHI [0]
Advanced Prediciton: CHI [0]
------------------------------
Normal Prediciton: DAL [0]
Advanced Prediciton: DAL [0]
------------------------------
Normal Prediciton: DEN [0]
Advanced Prediciton: DEN [1]
------------------------------
Normal Prediciton: GSW [1]
Advanced Prediciton: GSW [1]
------------------------------
Normal Prediciton: HOU [1]
Advanced Prediciton: HOU [1]
------------------------------
Normal Prediciton: LAC [0]
Advanced Prediciton: LAC [1]
------------------------------
Normal Prediciton: LAL [0]
Advanced Prediciton: LAL [0]
------------------------------
Normal Prediciton: MIA [0]
Advanced Predici